<a href="https://colab.research.google.com/github/dancher00/Control/blob/main/ACM_Seminar_2_Lyapunov_stability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prerequisites (run this)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint
plt.rcParams['figure.figsize'] = [8, 4]
plt.rcParams['animation.frame_format'] = "svg"
from matplotlib import rc
rc('animation', html='jshtml')

from matplotlib.animation import FuncAnimation


def plot_dynamics_with_lyapunov_function(x_dot, x_0, L,
                                         frames=100,
                                         time=1,
                                         axes_limits=[-1, 1, -0.1, 1]):
  plt.ioff()
  t = np.linspace(0, time, frames)
  def rhs(x, t):
    return x_dot(x)
  x = odeint(rhs, np.array([x_0]), t)[:, 0]
  trajectory = np.stack((x, L(x))).T
  if len(trajectory.shape) == 1:
      trajectory = trajectory.reshape(trajectory.size, 1)
  if trajectory.shape == (trajectory.size, 1):
      trajectory = np.stack((trajectory.T[0], np.zeros(trajectory.size))).T
  frames = range(trajectory.shape[0])
  fig, ax = plt.subplots(figsize=(8, 4))
  ax.axis(axes_limits)
  ax.set_aspect("equal")
  plt.grid()
  x = np.linspace(*axes_limits[:2], 1000)
  ax.plot(x, L(x), label="$L(x)$")
  point1, = ax.plot(0,1, marker="o", label="$(x(t), L(x(t)))$")
  point2, = ax.plot(0,1, marker="o", label="$(x(t), 0)$")
  ax.legend()
  ax.grid()
  ax.set_xlabel("$x$")

  # Updating function, to be repeatedly called by the animation
  def update(t):
      # obtain point coordinates
      x,y = trajectory[int(t) % trajectory.shape[0]]
      # set point's coordinates

      point1.set_data([x],[y])
      point2.set_data([x],[0])
      return point1, point2


  ani = FuncAnimation(fig, update, interval=1000/24, blit=True, repeat=True,
                    frames=frames)
  plt.ion()
  return ani

def plot_tanks_trajectory(trajectory, L = None):
    plt.ioff()
    frames = range(trajectory.shape[0])
    fig, ax = plt.subplots(figsize=(8, 4), ncols=1 if L is None else 2)
    if L is None:
      ax = [ax]
    barcollection = ax[0].bar(range(1, 101), trajectory[0])
    if L is not None:
        L_curve = [L(state) for state in trajectory]
        ax[1].axis([0, trajectory.shape[0], 0, np.max(L_curve)])
        L_plot, = ax[1].plot([0], L_curve[:1])
        ax[1].set_xlabel("Time")
        ax[1].set_ylabel("$L$")
        ax[1].grid()
    ax[0].set_xlabel("Tank index")
    ax[0].set_ylabel("Contained volume (m${}^3$)")
    # Updating function, to be repeatedly called by the animation
    def update(i):
        artists = []
        for j, b in enumerate(barcollection):
            b.set_height(trajectory[i, j])
            artists.append(b)
        if L is not None:
            L_plot.set_data(range(i), L_curve[:i])
            artists.append(L_plot)
        return artists


    ani = FuncAnimation(fig, update, interval=1000/24, blit=True, repeat=True,
                      frames=frames)
    plt.ion()
    return ani

# Lyapunov stability
## Recap

Let $\dot{x} = f(x)$.

$$
\newcommand{\nrm}[1]{\lVert #1 \rVert}
$$

$\textbf{Definition 1. Global asymptotic stability (GAS)}$:
$$
x(t) \xrightarrow{t → ∞} 0.
$$

$\textbf{Definition 2. Local asymptotic stability (LAS)}$:
$$
\nrm{x(0)} < ɛ \implies x(t) \xrightarrow{t → ∞} 0.
$$

$\textbf{Theorem 1. Lyapunov theorem}$:

The origin of the system $\dot{x} = f(x)$ is GAS if there exists such $L(x)$ that:
1. $L$ is positive-definite. (zero at the origin and positve everwhere else)
2. $L$ is radially unbounded. (diverges to $\infty$ in every direction)
3. $L$ is decaying. ($x(t)$ will move in such a way that $L$ decreases)

## A physical analogy to Lyapunov functions: Electric potential
![](https://d1whtlypfis84e.cloudfront.net/guides/wp-content/uploads/2019/09/28054528/Electric-Potential.png)


It is well known that a massless positively-charged particle will move in such a way that its electric potential constantly decreases.
$$
\dot{\phi} < 0
$$

Thus, if a negative charge is placed at the origin, the particle will approach the charge by descending the potential funnel formed by the negative charge.

With Lyapunov functions it's the exact same idea:
1. We require that our state (particle) descends the Lyapunov function (potential)
2. We require that the Lyapunov function (potential) is positive-definite and radially unbounded (has the shape of a funnel)
3. If such a function exists, then the state (particle) will approach the origin (negative charge).

Equivalentley 1, 2, 3 can be formulated rigorously in the following way:
1. Assume: $\frac{\partial }{\partial t}L(x(t)) < 0, \text{ when } x(t) \neq 0$.
2. Assume:
  * $L(x) > 0, \text{ when } x \neq 0, L(0) = 0$ (positive-definite)
  * If $\nrm{x_n} \xrightarrow{n \rightarrow \infty} \infty$, then $L(x_n) \xrightarrow{n \rightarrow \infty} \infty$ (radially unbounded)
3. Follows: $x(t) \xrightarrow{t \rightarrow \infty} 0$.

## Lyapunov stability: a simple example

$$
\dot{x} = -x,\\
L(x) = x^2.
$$

$$\frac{\partial}{\partial t}L(x(t)) = \nabla L(x(t)) \cdot \dot{x} = -2x^2 < 0 \ \ \ (\text{if }x \neq 0),\\
L(x) = x^2 > 0, x \neq 0. \\
L(0) = 0$$

In [ ]:
plot_dynamics_with_lyapunov_function(x_dot = lambda x: -x,
                                     x_0 = 1,
                                     L = lambda x: x**2
                                     )

## Excercise 1: Why do we need positive-definitness?

Why is it that the Lyapunov theorem wouldn't work if we were to relax positive-definitness?

1. Construct an example of an unstable one-dimensional system $\dot{x} = f(x)$ for which there exists a non-positive-definite "Lyapunov function". Demonstrate Lyapunov decay together with an absence of stability.
2. Why do we need positive-definitness? Provide an intuitive explanation.

To do this, you'll need to come up with $x_0$, $f(x)$ and $L(x)$ in such a way that it is visible (in the animation) that $L(x(t))$ is indeed decreasing, but at the same time $x(t)$ is not approaching $0$.

In [ ]:
plot_dynamics_with_lyapunov_function(x_dot = ..., # right hand side of the state dynamics equation for your system
                                     x_0   = ..., # initial state,
                                     L     = ..., # Lyapunov function
                                     axes_limits = ...
                                     )

## Excercise 2: Why do we need radial unboundedness?

Why is it that the Lyapunov theorem wouldn't work if we were to relax radial unboundedness?

1. Construct an example of an unstable one-dimensional system $\dot{x} = f(x)$ for which there exists a non-raidally-unbounded "Lyapunov function". Demonstrate Lyapunov decay together with an absence of stability.
2. Why do we need radial unboundedness? Provide an intuitive explanation.

To do this, you'll need to come up with $x_0$, $f(x)$ and $L(x)$ in such a way that it is visible (in the animation) that $L(x(t))$ is indeed decreasing, but at the same time $x(t)$ is not approaching $0$.

In [ ]:
plot_dynamics_with_lyapunov_function(x_dot = ..., # right hand side of the state dynamics equation for your system
                                     x_0   = ..., # initial state,
                                     L     = ..., # Lyapunov function
                                     axes_limits = ...
                                     )

## Excercise 3:
Let $L_1$ and $L_2$ be Lyapunov functions. Prove that their sum $L_1 + L_2$ and their product $L_1L_2$ will too be Lyapunov functions.

# Finding Lyapunov functions

## Trivial cases and common techniques



### Negative identity
$$\dot{x} =  -Ix,\\ L(x) = \nrm{x}_2^2.$$


### Symmetric linear systems
$$\dot{x} =  Ax,\\ L(x) = \nrm{Q^{-1}x}_2^2.$$
Where $Q$ is the [modal matrix](https://en.wikipedia.org/wiki/Modal_matrix) of $A$. More generally, this applies to all diagonalizable matrices $A$ with real eigenvalues (all matrices that are [similar](https://en.wikipedia.org/wiki/Matrix_similarity) to symmetric matrices).

### General linear systems

If $\dot{x} = Ax$, then $L(x) = x^TPx$, where $P$ is such that $A^TP + PA$ is negative-definite. The above result is better known as the continuous Lyapunov equation.

$P$ can in fact be computed explicitly by solving the following linear equation:
$$
(I - A \otimes A)\text{vec}(P) = -\text{vec}(Q),
$$
where $Q$ is an arbitrary symmetric positive-definite matrix. For instance the above would work for $Q = I$. Here $\text{vec}(X)$ denotes a vector composed of columns of matrix $X$ (a.k.a. [vectorization](https://en.wikipedia.org/wiki/Vectorization_(mathematics))), while $\otimes$ denotes the [kronecker product](https://en.wikipedia.org/wiki/Kronecker_product).

### Linearly conjugate to another system
As it turns out, if $\dot{x} = f(x)$ has a Lyapunov function $L(x)$, then the system $\dot{y} = Qf(Q^{-1}x)$ will have the Lyapunov function $L(Q^{-1}y)$. Note, that $y$ is the result of the following substituition $y := Qx$.

### Non-linearly conjugate to another system
You can in fact extend this principle to non-linear substituitions of the kind $y = \varphi(x)$, where $\varphi(x)$ is a differentiable one-to-one (bijective) map. In this case, the fact that $\dot{x} = f(x)$ has the Lyapunov function $L(x)$ implies that $\dot{y} = (\nabla \varphi)(\varphi^{-1}(y))f(\varphi^{-1}(y))$ has the lyapunov function $L(\varphi^{-1}(y))$. However non-linear conjugation of this kind is not a common technique to encounter.

### Collinear to another system
Let $\dot{x} = f(x)$ and $\dot{y} = g(y)$. And let it be that $g$ and $f$ are collinear (point in he same direction), i.e. one of the following three equivalent conditions holds:
1. $\forall x \ f(x)^Tg(x) = \nrm{f(x)}_2\nrm{g(x)}_2$
2. $f(x)\nrm{g(x)} = g(x)\nrm{f(x)}$
3. There exists such $\alpha(x) > 0$ that $f(x) = \alpha(x)g(x)$

Then $x$ and $y$ share the same Lyapunov function (if it exists).

### With insignificant non-linearity
Let's say you found a Lyapunov function for a linear system:
$$
\dot{x} = Ax,\\
L(x) = x^TPx
$$

Then this Lyapunov function $L$ will also be a Lyapunov function for the following system:
$$
\dot{y} = Ay + G(y)y,
$$
provided that $G(y)$ is bounded (i.e. $\forall y \ \nrm{G(y)}_F \leq G_\max$) and $\lambda_\max < -2 G_\max\nrm{P}_F$, where $\lambda_\max$ is the largest eigenvalue of $A^TP + PA$ and $\nrm{\cdot}_F$ denotes the [Frobenius norm](https://en.wikipedia.org/wiki/Matrix_norm#Frobenius_norm).

### Trial and error
Oftentimes you can find a Lyapunov function by assuming it to be a positive-definite quadratic form $x^TPx$ and then trying to derive the right choice of entries within $P$.

## Excercises on formal derivation (Section A)


### Excercise A.1
Determine a Lyapunov function for
$$
\dot{x} = -x^2\text{tanh}(x)
$$
$\textbf{Hint}$: Sometimes one can just guess the Lyapunov function and check if it fits.


### Excercise A.2
Determine a Lyapunov function for
$$
\dot{x} = -x^3,\\
\dot{y} = -yx^2 + x^3
$$
$\textbf{Hint}$: Try using one of those fancy techniques.

### Excercise A.3
Determine a Lyapunov function for.
$$
\dot{x} = e^{x^2}(y - x) + 0.001y + 0.001e^{(x-y)(x + y)}x,\\
\dot{y} = -e^{x^2}y + 0.001x + 0.001e^{(x-y)(x + y)}y
$$
$\textbf{Hint 1}$: You'll need to use two techniques to demonstrate that it has the same Lyapunov function as a certain linear system.

$\textbf{Hint 2}$: Do not use trial and error.

## Excecises on software implementation (Section B)



### B.1 Stability of linked water tanks

Consider a system of a 100 water tanks placed in such a way that the first tank is leaking into the second tank, the second tank is leaking into the third tank, ..., and the last tank is leaking into the ground. It is known that the rate at which water escapes a tank of this kind is equal to half of the volume of water currently contained in the tank. For instance if tank $i$ contains 10 liters of water, then water is escaping tank $i$ at a rate of 5 liters per second. If $i < 100$, then this water is flowing to tank $i + 1$, otherwise it simply drips to the ground and disappears.

$\textbf{Objective:}$ Prove that the tanks will always run dry.

$\textbf{Directions:}$
0. Figure out how to formally represent this system with ordinary differential equations.

1. Use [odeint](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html) to simulate the system. You should in this way be able to obtain trajectories of your system.

2. Then use [scipy's Lyapunov equation solver](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve_continuous_lyapunov.html) to determine a Lyapunov function.

3. Verify your results with `plot_tanks_trajectory(trajectory, L)`.


![](https://gitflic.ru/project/aidynamicaction/classedu2023-advctrl/blob/raw?file=img%2Ftanks.png&inline=false&commit=e2d719f1ad970a624f5ea31a993b89b04e4eb68b)





In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.linalg import solve_continuous_lyapunov

...

time=100
frames=100
t = np.linspace(0, time, frames)
def rhs(y, t):
    return ...
x_0 = np.random.random(size=100)**2
trajectory = ...

plot_tanks_trajectory(trajectory=trajectory,
                      L=...)